In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/drive/MyDrive/NepalEarhquakeDamage2015.csv")

print(df.columns)

# Drop specific columns
columns_to_drop = ['building_id', 'district_id', 'count_floors_post_eq','height_ft_post_eq', 'roof_type', 'condition_post_eq']
df.drop(columns_to_drop, axis=1, inplace=True)


df.dropna(inplace=True)

print(df.columns)
print(df.shape)

Index(['building_id', 'district_id', 'count_floors_pre_eq',
       'count_floors_post_eq', 'age_building', 'plinth_area_sq_ft',
       'height_ft_pre_eq', 'height_ft_post_eq', 'land_surface_condition',
       'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type',
       'position', 'plan_configuration', 'condition_post_eq', 'damage_grade'],
      dtype='object')
Index(['count_floors_pre_eq', 'age_building', 'plinth_area_sq_ft',
       'height_ft_pre_eq', 'land_surface_condition', 'foundation_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'damage_grade'],
      dtype='object')
(762094, 11)


In [46]:
# Kategorik sütunlar listesi
categorical_columns = ['land_surface_condition', 'foundation_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration']

# LabelEncoder nesnesi oluştur
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Daha sonra ters dönüşüm için sakla

# Hedef ve özellikler
X = df.drop('damage_grade', axis=1)
y = df['damage_grade']
num_classes = y.nunique()

# Hedef değişkeni sayısal değere dönüştür
le_y = LabelEncoder()
y = le_y.fit_transform(y)


# X ve y, ön işlemden geçmiş özellikler ve etiketlerdir
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(y)

[2 4 1 ... 4 4 4]


In [48]:

# Modeli oluştur
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Modeli derle
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Erken durdurma
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Modeli eğit
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/10
15242/15242 [==============================] - 57s 4ms/step - loss: 1.4174 - accuracy: 0.3912 - val_loss: 1.3950 - val_accuracy: 0.4006
Epoch 2/10
15242/15242 [==============================] - 55s 4ms/step - loss: 1.3837 - accuracy: 0.4050 - val_loss: 1.3869 - val_accuracy: 0.3923
Epoch 3/10
15242/15242 [==============================] - 55s 4ms/step - loss: 1.3794 - accuracy: 0.4065 - val_loss: 1.3842 - val_accuracy: 0.3901
Epoch 4/10
15242/15242 [==============================] - 55s 4ms/step - loss: 1.3754 - accuracy: 0.4076 - val_loss: 1.3687 - val_accuracy: 0.4102
Epoch 5/10
15242/15242 [==============================] - 55s 4ms/step - loss: 1.3736 - accuracy: 0.4086 - val_loss: 1.3732 - val_accuracy: 0.4040
Epoch 6/10
15242/15242 [==============================] - 56s 4ms/step - loss: 1.3731 - accuracy: 0.4089 - val_loss: 1.3785 - val_accuracy: 0.3990
Epoch 7/10
15242/15242 [==============================] - 56s 4ms/step - loss: 1.3724 - accuracy: 0.4090 - val_loss: 1

In [49]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# TFLite modelini dosyaya yaz
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [7]:
X_train.head()
print(X_train.shape)

(609675, 50)


In [10]:
from joblib import dump, load
model_path = '/content/drive/My Drive/model.pkl'  # Google Drive'da kaydetmek istediğiniz yol
dump(model, model_path)

['/content/drive/My Drive/model.pkl']

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
